# Objective

Answer the question:

* What is the relationship between observations recorded as "head counts" and those recorded as "sweeps"?

Document the process of finding the answer so that the exact steps are replicable and clearly demonstrated here.

## Background

!["Perfectly straight line in graph comparing supposedly independent variables in Head Count and Sweeps workbook, in Excel Online"](head-count-sweeps-graph-excel-online.png)

I'm troubleshooting a spreadsheet document that was prepared by someone else. Whoever prepared it probably handed it off to someone else before I came into contact. The information in the document looks like it's been copied from other documents which I'm not certain I have access to.

In this document, there's a graph that demonstrates the goal of the document: to compare the relationship between two observation methods. Unfortunately, the graph shows a suspicious degree of idealness: a perfect one-to-one ratio across the entire domain.

My assignment is to trace the error and correct the graph so that it displays the precise ratios calculated from appropriate samples.

## Expected Results

Upon plotting the ratio of numbers of specimens by each of two collection methods, the graph should suggest a trend that is not perfectly linear.

# Procedure

## Setup

* Document handling and analysis will be conducted in [Jupyter] Notebook/Lab, using [Python] 3.
* Microsoft [Office] Excel Online will be used to create and embed a graph. The offline version would also suffice.
* To setup a live workspace, consult the [README] for the home project comprising this document and its associated files.
* If your live workspace doesn't include it, [install _pandas_] before continuing. The commonly recommended way to do this is with `pip install pandas`. (If you're using [Anaconda], you probably already have _pandas_.)

[Anaconda]: http://docs.continuum.io/anaconda/
[Jupyter]: https://jupyter.org/
[Office]: https://www.office.com/
[Python]: https://www.python.org/about/
[README]: https://github.com/devvyn/aafc-field-data/blob/master/README.md
[install _pandas_]: https://pandas.pydata.org/pandas-docs/stable/install.html

In [69]:
!pip install pandas

* Direct access to the [file] `2016-sweep-vs-tiller.xlsx` is required in the live workspace. Available to be [downloaded][file] from GitHub.

[file]: https://github.com/devvyn/aafc-field-data/blob/master/notebook/projects/2016-sweep-vs-tiller/2016-sweep-vs-tiller.xlsx

## Import Data

In [70]:
import pandas


data_file = pandas.ExcelFile('2016-sweep-vs-tiller.xlsx')
sheets = {
    sheet_name: data_file.parse(sheet_name)
    for sheet_name in data_file.sheet_names
}

## Explore Worksheets

When I opened the workbook in Excel Online, I saw many sheets with rather unhelpful names, and what looked like a lot of data that had been copied from other worksheets.

### Two Sources of Observational Data

Data sets to compare:

* "cereal sweeps" or just "sweeps"
* "head counts" or "tillers"

### Unbelievable Graph

- There's a graph in a worksheet called "head counts vs sweeps graphs" which demonstrates the analytical problem encountered/developed by someone else.
- The data supposedly being compared in the graph is cannot be the data that was intended for comparison because the ratio depicted is perfectly linear even though it's comparing real world samples.

### Lack of Spreadsheet Formulas

It's clear that the Excel workbook has the results of many calculations, yet there are no formula cells. In order to check the accuracy of the calculations, I need to replicate them from scratch.

### Ambiguously Duplicated Data

- Data may have been copied from multiple, unidentified sources.
- Multiple editors have made changes or additions to the workbook, and nobody left notes.
- It's not clear which data is "original" and which is duplicated, amongst the worksheets in the workbook.

### Compare Columns

To discern which data is original, I'll begin by listing the columns of all sheets, which will offer some descriptive terms for the data in each:

In [71]:
pandas.DataFrame(
    data=[frame.columns for frame in sheets.values()],
    index=sheets.keys()
)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
Head Counts,Site,Crop,Date,Field,Zadoks_stage,Tiller,EGA_head,EGA_leaf,BCO_head,BCO_leaf,...,None,None,None,None,None,None,None,None,None,None
Sheet2,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name,Crop,...,Hymenoptera_Figitidae,Hymenoptera_Aphelinidae,Hymenoptera_Perilampidae,Hymenoptera_Chalcidoidea,Hymenoptera_Ichneumondoidea,Hymenoptera_Proctotrupoidea,None,None,None,None
Sweep Samples Cereals,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name,Crop,...,Hymenoptera_Proctotrupidae,Hymenoptera_Pteromalidae,Hymenoptera_Apidae,Hymenoptera_Diplazontinae,Hymenoptera_Figitidae,Hymenoptera_Aphelinidae,Hymenoptera_Perilampidae,Hymenoptera_Chalcidoidea,Hymenoptera_Ichneumondoidea,Hymenoptera_Proctotrupoidea
Head Counts Edited,Date,Site,Crop,Field,Sample Type,Unnamed: 5,Zadoks_stage,Tiller,EGA_alate,EGA_apt,...,None,None,None,None,None,None,None,None,None,None
Sweep Samples Cereals Edited,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Unnamed: 6,Unnamed: 7,EGA_alate,EGA_apt,...,None,None,None,None,None,None,None,None,None,None
Data Sheets Combined,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller,EGA_alate,...,None,None,None,None,None,None,None,None,None,None
Pivot chart LH phen,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
leafhoppers 2016 cereal sweeps,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name,Crop,Distance(m),Number of Samples,...,None,None,None,None,None,None,None,None,None,None
Sheet3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
aphid sweep vs head count,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller,EGA_alate,...,None,None,None,None,None,None,None,None,None,None


### Initial Grouping

Based on sheet names, column names, and similarities between columns sets, I can probably group the sheets like so:

- Head counts:
  - Head Counts
  - Head Counts Edited
- Sweep:
  - Sheet2
  - Sweep Samples Cereals
  - Sweep Samples Cereals Edited
  - leafhoppers 2016 cereal sweeps
- United:
  - Data Sheets Combined
- Analytical experiments:
  - Pivot chart LH phen
  - Sheet3
  - aphid sweep vs head count
  - head counts vs sweeps graphs

Because I can't trust the accuracy of the data used in the graph, I need to look at all the sheets and determine the most complete and unadulterated data sets. I'll determine which data belongs to each category, and compare the sets.

## Select Primary Data Set: Sweep

- Sheet2
- Sweep Samples Cereals
- Sweep Samples Cereals Edited
- leafhoppers 2016 cereal sweeps

### Sweep Samples Cereals vs Sheet2

I'd like to confirm that the vaguely named **Sheet2** is what it seems to be: a slightly edited copy of **Sweep Samples Cereals**.

#### Columns

In [72]:
sheet_names = [
    'Sweep Samples Cereals',
    'Sheet2',
]
compare_sheets = ssc, s2 = [
    sheets[sheet_name]
    for sheet_name in sheet_names
]
pandas.DataFrame(
    data=[sheet.columns for sheet in compare_sheets],
    index=sheet_names
).T

,Sweep Samples Cereals,Sheet2
0,ID,ID
1,Province,Province
2,Collection_Date,Collection_Date
3,Sample_by_week,Sample_by_week
4,Date_by_week,Date_by_week
5,Date,Date
6,Julian_date,Julian_date
7,Site,Site
8,Field_name,Field_name
9,Crop,Crop


That seems to confirm that **Sheet2** has the same columns as **Sweep Samples Cereals** except for four columns that were removed.

Compare lists of column names:

In [73]:
set.symmetric_difference(*(
    set(sheet.columns.tolist())
    for sheet in compare_sheets
))

{'EGA/20 Sweeps',
 'EGA/Sweep',
 'Total EGA',
 'Total Sweeps',
 'Unnamed: 129',
 'Unnamed: 133'}

The unnamed columns are inconsequential to our analysis. In fact, I believe they're empty. The rest have aggregate values that I don't trust. **Sweep Samples Cereals** is probably useless for my purposes.

#### Rows

What about the rows? I'll check sizes:

In [74]:
pandas.DataFrame(
    data=[
        sheets[sheet_name].index.size
        for sheet_name in sheet_names
    ],
    index=sheet_names,
)

,0
Sweep Samples Cereals,92
Sheet2,668


It's pretty clear there is a lot more data in **Sheet2**. I suspect that **Sheet2** has additional data added to it. I'll have to take a closer look at the values, especially dates.

Before I can easily examine dates in the worksheets, I should convert them to proper datetime values:

In [75]:
for sheet in compare_sheets:
    sheet['Collection_Date'] = pandas.to_datetime(
        sheet['Collection_Date'],
        format='%d_%m_%Y',
    )

Which sheet covers the earliest dates?

In [76]:
[sheet['Collection_Date'].min()
 for sheet in compare_sheets]

[Timestamp('2016-06-05 00:00:00'), Timestamp('2016-06-05 00:00:00')]

Neither.

Which sheet covers the latest dates?

In [77]:
[sheet['Collection_Date'].max()
 for sheet in compare_sheets]

[Timestamp('2017-07-08 00:00:00'), Timestamp('2017-07-08 00:00:00')]

Neither, again.

Are all the dates used in both?

In [78]:
[len(sheet['Collection_Date'].unique())
 for sheet in compare_sheets]

[32, 34]

No, not exactly.

I need to check if all the dates in **Sweep Samples Cereals** are in **Sheet2**.

In [79]:
ssc.Collection_Date.isin(s2.Collection_Date).all()

True

**Sweep Samples Cereals** is most likely either a subset of **Sheet2**, or a reduced version of the same source data. **Sheet2** actually has more unique dates. Therefore, some dates in **Sheet2** must be absent from **Sweep Samples Cereals**.

In [80]:
s2.Collection_Date.isin(ssc.Collection_Date).all()

False

#### Aggregation

So, both sheets cover the same date range. **Sheet2** has two additional dates. I'm curious if one is aggregated from the other, especially since I see a "total sweeps" column in **Sweep Samples Cereals**.

In [81]:
ssc['Total Sweeps'].head()

0    120
1    120
2     60
3    120
4     80
Name: Total Sweeps, dtype: int64

I suspect that the sweeps were at different distances, and later reduced to sums. If I peek at the `Distance(m)` column, I should see a clear difference.

In [82]:
ssc['Distance(m)'].unique()

array(['Combined'], dtype=object)

In [83]:
s2['Distance(m)'].unique()

array([0, 5, 10, 25, 50, 100, 'Combined'], dtype=object)

Indeed, **Sheet2** has observations at various "distances", while **Sweep Samples Cereals** has only the label, "Combined".

**Sheet2** has more rows because it isn't totalling up the sweeps from various distances. That makes **Sheet2** less reduced, and more "raw".
Since I don't want reduced (aggregated) data, I don't want **Sweep Samples Cereals**.

#### Conclusion

Optimal candidate:

- **Sheet2**

### Sweep Samples Cereals Edited vs Sheet2

In [84]:
ssce = sheets['Sweep Samples Cereals Edited']
compare_sheets = ssce, s2

#### Columns

In [85]:
pandas.options.display.max_rows = 140
pandas.DataFrame(
    data=[sheet.columns for sheet in compare_sheets],
    index=['Sweep Samples Cereals Edited', 'Sheet2']
).T

,Sweep Samples Cereals Edited,Sheet2
0,Date,ID
1,Site,Province
2,Crop,Collection_Date
3,Field_name,Sample_by_week
4,Sample Type,Date_by_week
5,Total Sweeps,Date
6,Unnamed: 6,Julian_date
7,Unnamed: 7,Site
8,EGA_alate,Field_name
9,EGA_apt,Crop


**Sweep Samples Cereals Edited** seems to have left out some columns that would be expected to carry finely categorized subjects, such as various instars of aphids and leafhoppers. This makes it less likely to have information I'll need. 

Furthermore, it appears that **Sweep Samples Cereals Edited** has a column that is probably an artefact from aggregation: `Total Sweeps`. It's also missing the `Distance(m)` column; another sign of aggregation, and therefore loss of some information.

I hope I can ignore the whole worksheet, just as I'll be doing with **Sweep Samples Cereals** ("unedited"). Unless further analysis reveals that **Sweep Samples Cereals Edited** has dates that are missing from **Sheet2**, I'll assume it's not worth closer examination.

#### Rows

In [86]:
f'ssce has {ssce.index.size / s2.index.size:.0%} of the length of s2'

'ssce has 14% of the length of s2'

**Sweep Samples Cereals Edited** has 14% the number of rows as **Sheet2**, so it's not likely to be useful, unless the dates don't fully overlap.

In [87]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


descriptors = [pandas.Series.max, pandas.Series.min, len_unique]
ssce.Date.apply(descriptors)

max           2016-08-15 00:00:00
min           2016-06-05 00:00:00
len_unique                     31
Name: Date, dtype: object

In [88]:
s2.Collection_Date.apply(descriptors)

max           2017-07-08 00:00:00
min           2016-06-05 00:00:00
len_unique                     34
Name: Collection_Date, dtype: object

It appears that **Sweep Samples Cereals Edited**, like **Sweep Samples Cereals**, has a much shorter date range, so we won't be missing anything if we ignore it. To be sure, I need to check if all the dates in **Sweep Samples Cereals Edited** are in **Sheet2**.

In [89]:
ssce.Date.isin(s2.Collection_Date).all()

True

Excellent. I see no reason to pay attention to **Sweep Samples Cereals Edited** or **Sweep Samples Cereals** anymore. 

If I have time after fixing the graph, I may trace the cause of the error, which may lead me back to one of those worksheets.

#### Conclusion

Optimal candidate:

- **Sheet2** (again)

### leafhoppers 2016 cereal sweeps vs Sheet2

In [90]:
sheet_names = ['leafhoppers 2016 cereal sweeps', 'Sheet2']
compare_sheets = lh2016, s2 = [
    sheets[sheet_name]
    for sheet_name in sheet_names
]

#### Columns

In [91]:
columns = [frame.columns for frame in compare_sheets]
pandas.DataFrame(
    data=columns,
    index=sheet_names
).T.head(lh2016.columns.size)

,leafhoppers 2016 cereal sweeps,Sheet2
0,Collection_Date,ID
1,Sample_by_week,Province
2,Date_by_week,Collection_Date
3,Date,Sample_by_week
4,Julian_date,Date_by_week
5,Site,Date
6,Field_name,Julian_date
7,Crop,Site
8,Distance(m),Field_name
9,Number of Samples,Crop


Clearly, **leafhoppers 2016 cereal sweeps** is focused on leafhoppers. Because the object of our analysis is to compare aphid numbers, I don't see the relevance of this leafhopper data.

In [92]:
lh2016['Total Sweeps'].unique()

array([120,  60,  80,  20, 100,  40])

In [93]:
lh2016['Distance(m)'].unique()

array(['Combined'], dtype=object)

In [94]:
lh2016['Number of Samples'].unique()

array([6, 3, 4, 1, 5, 2])

#### Rows

In [95]:
pandas.DataFrame(data={sheet_name: sheet.index.size for sheet_name, sheet in zip(sheet_names, compare_sheets)}, index=['number of rows']).rename_axis(['worksheet'], axis='columns').T.sort_values(by='number of rows')

,number of rows
worksheet,
leafhoppers 2016 cereal sweeps,92
Sheet2,668


Sheet2 has the most rows. But do the times align?

First, what date format is used in lh2016?

In [96]:
lh2016.Collection_Date.head()

0    12_08_2016
1    05_08_2016
2    08_07_2017
3    15_07_2016
4    15_07_2016
Name: Collection_Date, dtype: object

Seems like a text expression. Day, month, year; separated by underscores. The function `pandas.to_datetime` solves this.

In [97]:
lh2016.Collection_Date = pandas.to_datetime(lh2016.Collection_Date, format='%d_%m_%Y')
lh2016.Collection_Date.head()

0   2016-08-12
1   2016-08-05
2   2017-07-08
3   2016-07-15
4   2016-07-15
Name: Collection_Date, dtype: datetime64[ns]

Now, to compare dimensions:

In [98]:
pandas.DataFrame(
    data=[
        len_unique(frame.Collection_Date.index)
        for frame in compare_sheets
    ],
    index=sheet_names,
    columns=[
        'unique datetimes',
    ],
)

,unique datetimes
leafhoppers 2016 cereal sweeps,92
Sheet2,668


Seeing that `leafhoppers 2016 cereal sweeps` is smaller, check that its index is a subset of `Sheet2`:

In [99]:
lh2016.Collection_Date.index.isin(s2.index).all()

True

All of the dates in the leafhopper counts are present in `Sheet2`.

#### Conclusion

Given that the leafhopper data isn't pertinent, `Sheet2` remains the best candidate for the primary source of data points about aphids collected and counted according to the "sweep" method.

If it's ultimately determined that the data *is* relevant, it may be useful in that case because the datetime index overlaps with that of `Sheet2`.

### Conclusion

After all analysis, `Sheet2` appears to be the purest, most relevant base for comparison of "sweep" sample data to that of "tiller" count data.

## Select Primary Data Set: Tiller Head

- Head Counts
- Head Counts Edited

### Head Counts vs Head Counts Edited

I presume the relationship between these two worksheets is the same as that between the equivalent "sweep" worksheets. Therefore, I expect the "edited" version to be less useful.

In [100]:
sheet_names = [
    'Head Counts',
    'Head Counts Edited'
]
compare_sheets = hc, hce = [sheets[sheet_name] for sheet_name in sheet_names]

#### Columns

In [101]:
pandas.DataFrame(
    index=sheet_names,
    data=[sheet.columns for sheet in compare_sheets],
).T

,Head Counts,Head Counts Edited
0,Site,Date
1,Crop,Site
2,Date,Crop
3,Field,Field
4,Zadoks_stage,Sample Type
5,Tiller,Unnamed: 5
6,EGA_head,Zadoks_stage
7,EGA_leaf,Tiller
8,BCO_head,EGA_alate
9,BCO_leaf,EGA_apt


The main difference here seems to be the aggregation in the "edited" sheet, indicated by the columns named "EGA/head", "EGA_total", etc. I expect the "unedited" data to be more complete and reliable.

#### Rows

In [102]:
hce.Date.head()

0   2016-08-04
1   2016-08-04
2   2016-07-28
3   2016-08-12
4   2016-07-15
Name: Date, dtype: datetime64[ns]

In [103]:
hc.Date.head()

0    04/08/2016
1    04/08/2016
2    28/07/2016
3    12/08/2016
4    15/07/2016
Name: Date, dtype: object

I'll fix the date format with `pandas.to_datetime` again.

In [104]:
hc.Date = pandas.to_datetime(hc.Date,
                             format='%d/%m/%Y')
hc.Date.head()

0   2016-08-04
1   2016-08-04
2   2016-07-28
3   2016-08-12
4   2016-07-15
Name: Date, dtype: datetime64[ns]

Now, compare for completeness:

In [105]:
[len_unique(column) for column in (
    hc.Date,
    hce.Date
)]

[20, 20]

In [106]:
hc.Date.index.isin(hce.Date.index).all()

True

Identical date & time for the index of each, so no basis for choosing one over the other.

### Conclusion

Based on the columns, the best candidate for pure, reliable data is:

- `Head Counts`

## Align Primary Data Sets

The names and corresponding data frames, from the worksheets in the source workbook document (Excel):

In [153]:
sheet_names, compare_sheets = zip(
    ('Head Counts', hc),
    ('Sheet2', s2),
)

#### Illuminating Whitespace

For the sake of visualization, I'll write a function that wraps any value in square brackets.

In [146]:
def wrap_brackets(x):
    return f'[{str(x)}]'

### Compare Columns

I've already noticed at least one leading space on a column name ("spiders"), and variations in capitalization. To compensate for this, I'll strip all leading and trailing whitespace from the lower case column names.

Comparing alphabetically sorted column names between our two primary data sets:

In [156]:
for sheet in compare_sheets:
    sheet.columns = sheet.columns.str.lstrip().str.strip().str.lower()
    sheet.sort_index(axis='columns', inplace=True)

In [172]:
pandas.DataFrame(
    index=sheet_names,
    data=[sheet.columns for sheet in compare_sheets]
).fillna('').T

,Head Counts,Sheet2
0,aphid_mummies_blk,( damsel bug)nabis_americoferus_adult
1,aphid_mummies_brown,1st_instar_ega
2,aphids_total,1st_instar_macrosteles
3,bco_alate,2nd_instar_ega
4,bco_apt,2nd_instar_macrosteles
5,bco_head,3rd_instar_ega
6,bco_leaf,3rd_instar_ega_pre-alate
7,bco_total,3rd_instar_macrosteles
8,comments,4th_instar_macrosteles
9,crop,4th_instar_pre-alate


Wow, that's quite a large difference in columns for these sets. Since the object of the comparison is aphids only, we can ignore most of these columns from `Sheet2`. For better comparison, let's filter for columns referring to aphids.

#### Aphid Columns

Aphid related terms:

* aphid
* ega
* bco
* greenbug

In [174]:
aphid_terms = (
    r'aphids?',
    r'ega',
    r'bco',
    r'greenbug',
)
aphid_term_pattern = '|'.join(aphid_terms)

hc_aphid_columns, s2_aphid_columns = [
    sheet.columns[
        sheet.columns.str.contains(aphid_term_pattern)
    ] for sheet in compare_sheets
]

sorted(hc_aphid_columns.tolist() + s2_aphid_columns.tolist())

['1st_instar_ega',
 '2nd_instar_ega',
 '3rd_instar_ega',
 '3rd_instar_ega_pre-alate',
 'aphid_mummies',
 'aphid_mummies_aphelinus_black',
 'aphid_mummies_aphidius_brown',
 'aphid_mummies_blk',
 'aphid_mummies_brown',
 'aphidencyrtus_sp',
 'aphidiius_sp.',
 'aphids_total',
 'bco_alate',
 'bco_apt',
 'bco_head',
 'bco_leaf',
 'bco_total',
 'bird_cherry_oat_aphid',
 'ega alate',
 'ega_alate',
 'ega_apt',
 'ega_grn',
 'ega_head',
 'ega_leaf',
 'ega_red',
 'ega_total',
 'greenbug_alate',
 'greenbug_aphid',
 'greenbug_apt',
 'pea aphids',
 'sitobion_avenae_ega_green (wingless)',
 'sitobion_avenae_ega_red',
 'total_alate_aphids',
 'total_apterous_aphids']

I see some problems with this list.

Not aphid related:

- aphidencyrtus_sp
- aphidiius_sp
- aphid_mummies
- aphid_mummies_aphelinus_black
- aphid_mummies_aphidius_brown
- aphid_mummies_blk
- aphid_mummies_brown

Not primary data:

- aphids_total
- bco_total
- ega_total
- total_alate_aphids
- total_apterous_aphids

I can prevent the matching of words containing "aphid" by adding a word boundary definition:

In [111]:
boundary = r'(?:_|^|$|\b)'
aphid_term_pattern = ''.join((
    boundary,
    r'(?:', '|'.join(aphid_terms), r')',
    boundary,
))

hc_aphid_columns, s2_aphid_columns = [
    sheet.columns[
        sheet.columns.str.contains(aphid_term_pattern)
    ] for sheet in compare_sheets
]

sorted(hc_aphid_columns.tolist() + s2_aphid_columns.tolist())

['1st_instar_ega',
 '2nd_instar_ega',
 '3rd_instar_ega',
 '3rd_instar_ega_pre-alate',
 'aphid_mummies',
 'aphid_mummies_aphelinus_black',
 'aphid_mummies_aphidius_brown',
 'aphid_mummies_blk',
 'aphid_mummies_brown',
 'aphids_total',
 'bco_alate',
 'bco_apt',
 'bco_head',
 'bco_leaf',
 'bco_total',
 'bird_cherry_oat_aphid',
 'ega alate',
 'ega_alate',
 'ega_apt',
 'ega_grn',
 'ega_head',
 'ega_leaf',
 'ega_red',
 'ega_total',
 'greenbug_alate',
 'greenbug_aphid',
 'greenbug_apt',
 'pea aphids',
 'sitobion_avenae_ega_green (wingless)',
 'sitobion_avenae_ega_red',
 'total_alate_aphids',
 'total_apterous_aphids']

Better. Still need to exclude "total" and "mummies".

In [112]:
excluded_terms = (
    r'mumm(?:y|ies)',
    r'total',
)
aphid_term_pattern, excluded_term_pattern = (
    r''.join((
        boundary,
        r'(?:', '|'.join(pattern), r')',
        boundary,
    )) for pattern in (aphid_terms, excluded_terms)
)

hc_aphid_columns, s2_aphid_columns = [
    sheet.columns[
        sheet.columns.str.contains(aphid_term_pattern) & ~ sheet.columns.str.contains(excluded_term_pattern)
    ] for sheet in compare_sheets
]

sorted(hc_aphid_columns.tolist() + s2_aphid_columns.tolist())

['1st_instar_ega',
 '2nd_instar_ega',
 '3rd_instar_ega',
 '3rd_instar_ega_pre-alate',
 'bco_alate',
 'bco_apt',
 'bco_head',
 'bco_leaf',
 'bird_cherry_oat_aphid',
 'ega alate',
 'ega_alate',
 'ega_apt',
 'ega_grn',
 'ega_head',
 'ega_leaf',
 'ega_red',
 'greenbug_alate',
 'greenbug_aphid',
 'greenbug_apt',
 'pea aphids',
 'sitobion_avenae_ega_green (wingless)',
 'sitobion_avenae_ega_red']

Great! Aphid columns identified.

What remains?

#### Non-aphid Related Columns

In [113]:
hc_remainder, s2_remainder = (
    frame.columns[~frame.columns.isin(frame_aphid)]
    for frame, frame_aphid in zip((hc, s2), (hc_aphid_columns, s2_aphid_columns))
)

sorted(hc_remainder.tolist() + s2_remainder.tolist())

['( damsel bug)nabis_americoferus_adult',
 '1st_instar_macrosteles',
 '2nd_instar_macrosteles',
 '3rd_instar_macrosteles',
 '4th_instar_macrosteles',
 '4th_instar_pre-alate',
 'ambush_bugs',
 'anthocoridae',
 'anthomyiidae-delia',
 'any parasitoid_adults',
 'aphelinus_albipodus',
 'aphelinus_asychis',
 'aphelinus_varipes',
 'aphid_mummies',
 'aphid_mummies_aphelinus_black',
 'aphid_mummies_aphidius_brown',
 'aphid_mummies_blk',
 'aphid_mummies_brown',
 'aphidencyrtus_sp',
 'aphidiius_sp.',
 'aphids_total',
 'asaphes_suspensus',
 'assassin_bug (reduviid bugs)',
 'athysanus_argentarius',
 'bco_total',
 'bees',
 'beetles',
 'bertha_armyworms',
 'braconid_wasps',
 'cabbage_butterfly',
 'capsus_simulans',
 'caterpillar',
 'chalcid_wasps',
 'chinch_bug',
 'chrysopa_oculata_adult',
 'chrysopa_oculata_larvae',
 'chrysoperla_carnea_adult',
 'chrysoperla_carnea_larva',
 'chrysopidae_adults',
 'cicindela',
 'coccinella_septempunctata_c7',
 'collection_date',
 'comments',
 'crop',
 'crop',
 'date'

#### Independent Variable Columns

Reading through the list of remaining, non-aphid related columns, I see some that don't mention any organism by name. These columns may be useful for indexing, which is crucial to aligning the two data sources.

In [114]:
independent_variable_names = pandas.Series(data=(
    'collection_date',
    'comments',
    'crop',
    'date',
    'date_by_week',
    'distance(m)',
    'field',
    'field_name',
    'id',
    'julian_date',
    'number of samples',
    'province',
    'sample_by_week',
    'site',
    'zadoks_stage',
))

Here are the names of the matching columns from each frame:

In [115]:
independent_variable_common = pandas.DataFrame(
    {
        name: dict(zip(independent_variable_names,
                       independent_variable_names.isin(frame.columns)))
        for name, frame
        in zip(sheet_names, compare_sheets)
    }
).replace(to_replace={False: '', True: '✅'})
independent_variable_common

,Head Counts,Sheet2
collection_date,,✅
comments,✅,
crop,✅,✅
date,✅,✅
date_by_week,,✅
distance(m),,✅
field,✅,
field_name,,✅
id,,✅
julian_date,,✅


### Align Columns

Our next goal is to determine which columns are in common and ensure they're of the same data type, so we can concatenate the frames.

In [116]:
independent_variable_common.index[
    independent_variable_common.all(axis='columns')
].tolist()

['crop', 'date', 'site']

Those columns alone are probably enough to align the data.

These columns warrant examination as well:

- collection_date
- date
- distance(m)
- field
- field_name
- number of samples

#### Site & Field Name

The columns `hc.field` and `s2.field_name` seem to relate to `site` and `crop`. I suspect redundancy. Here's why I feel that way:

In [148]:
pandas.concat(
    (hc[['site', 'crop', 'field',]], s2[['site', 'crop', 'field_name',]]),
    axis='columns',
    keys=sheet_names,
).applymap(wrap_brackets).head()

Head Counts                                    Sheet2           \
             site      crop                 field      site     crop   
0  [Yellow Creek]   [Wheat]  [Yellow_Creek_Wheat]  [Alvena]  [Wheat]   
1  [Yellow Creek]   [Wheat]  [Yellow_Creek_Wheat]  [Alvena]  [Wheat]   
2  [Yellow_Creek]   [Wheat]  [Yellow_Creek_Wheat]  [Alvena]  [Wheat]   
3  [Yellow_Creek]   [Tanzy]  [Yellow_Creek_Tanzy]  [Alvena]  [Wheat]   
4         [Wakaw]  [Barley]        [Wakaw_Barley]  [Alvena]  [Wheat]   

                  
      field_name  
0  [AlvenaWheat]  
1  [AlvenaWheat]  
2  [AlvenaWheat]  
3  [AlvenaWheat]  
4  [AlvenaWheat]

It's clear to my eyes that for both sources, `site` and `crop` are concatenated to form `field` or `field_name`.

- site + crop = field(_name)

Primary data:

- site
- crop

Aggregated data:

- field
- field_name

Therefore, I can safely disregard those derived columns and rely on the the more normalized forms for indexing. That is to say I think it's most beneficial to use `crop` and `site`.

##### Unique Values for Site

In [145]:
s2.site.drop_duplicates().apply(wrap_brackets)

0            [Alvena]
174          [Clavet]
177     [Indian Head]
187          [Kernan]
193         [Kernan ]
295       [Llewellyn]
352         [Melfort]
363        [Melfort ]
428         [Outlook]
478             [SEF]
578           [Wakaw]
626    [Yellow creek]
632     [Yellowcreek]
639    [Yellow Creek]
Name: site, dtype: object

#### Crop

Since I've noticed some sneaky whitespace, I'll wrap the `crop` values in brackets.

The `crop` field values for both data frames:

In [121]:
crops = pandas.concat(
    (
        frame.crop.apply(str).drop_duplicates().sort_values()
        for frame in compare_sheets
    ),
    keys=sheet_names,
    sort=True,
)
crops.apply(wrap_brackets)

Head Counts  8                  [0]
             4             [Barley]
             120          [Barley ]
             84              [Oats]
             3              [Tanzy]
             0              [Wheat]
             101           [Wheat ]
             128     [Winter Wheat]
             185     [Winter_wheat]
             168           [barley]
             81               [nan]
             85          [unlisted]
             9              [wheat]
Sheet2       80            [Barley]
             94           [Barley ]
             404              [Oat]
             394             [Oat ]
             380             [Oats]
             0              [Wheat]
             66            [Wheat ]
             219     [Winter Wheat]
             239    [Winter Wheat ]
             73       [WinterWheat]
             295            [wheat]
Name: crop, dtype: object

Clearly, there are some variations that should be corrected, in both data frames.

- whitespace
- letter case
- word separation

I'll write a function that transforms any given "crop" value into a uniform representation of the crop it's intended to represent.

In [123]:
import re

def normalize_str(value, separator=' '):
    if value is pandas.np.nan:
        return value

    str_value = str(value)

    is_mixed_case = str_value.upper() != str_value.lower() and not (str_value.islower() or str_value.isupper())
    if is_mixed_case:
        word_index = [
            index for index, char in enumerate(str_value) 
            if char.isupper()
        ] + [None]
        if word_index:
            words = [
                str_value[word_index[i]:word_index[i + 1]].strip()
                for i in range(len(word_index) - 1)
            ]
            str_value = separator.join(words)
    
    transformed = re.compile(r'[^a-zA-Z0-9]').sub(separator, str(str_value).title())
    
    if transformed.endswith('Oats'):
        transformed = transformed[:-1]
    
    return transformed

Previewing the results, with the square brackets again:

In [124]:
pandas.concat(
    (
        crops.apply(wrap_brackets),
        crops.apply(normalize_str).apply(wrap_brackets),
    ),
    keys=("Before", "After"),
    sort=True,
    axis='columns',
)

Before           After
Head Counts 0            [Wheat]         [Wheat]
            3            [Tanzy]         [Tanzy]
            4           [Barley]        [Barley]
            8                [0]             [0]
            9            [wheat]         [Wheat]
            81             [nan]           [Nan]
            84            [Oats]           [Oat]
            85        [unlisted]      [Unlisted]
            101         [Wheat ]         [Wheat]
            120        [Barley ]        [Barley]
            128   [Winter Wheat]  [Winter Wheat]
            168         [barley]        [Barley]
            185   [Winter_wheat]  [Winter Wheat]
Sheet2      0            [Wheat]         [Wheat]
            66          [Wheat ]         [Wheat]
            73     [WinterWheat]  [Winter Wheat]
            80          [Barley]        [Barley]
            94         [Barley ]        [Barley]
            219   [Winter Wheat]  [Winter Wheat]
            239  [Winter Wheat ]  [Winter Wheat]
            295          [wheat]         [Wheat]
            380           [Oats]           [Oat]
            394           [Oat ]           [Oat]
            404            [Oat]           [Oat]

Apply the rename:

In [53]:
for frame in compare_sheets:
    frame.crop = frame.crop.apply(normalize_str)

#### Number of Samples & Distance

##### Number of Samples

This field is only present in `Sheet2`.

In [125]:
s2['number of samples'].value_counts(dropna=False)

NaN     576
 5.0     36
 6.0     31
 4.0      9
 3.0      8
 1.0      5
 2.0      3
Name: number of samples, dtype: int64

There are many missing values in this column, which — like the "Combined" value in the `distance(m)` column — suggests a mixture of data from multiple sources, some aggregated and some not. As with the distance column, the irregularities indicated here will affect the calculation of ratios if there's more than one value for a given date and place.

For the sake of efficiency, I'll focus on the `distance(m)` column when analyzing.

In [126]:
s2_indexed = s2.set_index(['site', 'crop', 'collection_date'])

In [127]:
s2_indexed[s2_indexed['number of samples'].isna()].head()

( damsel bug)nabis_americoferus_adult  \
site   crop  collection_date                                          
Alvena Wheat 2016-08-12                                         0.0   
             2016-08-12                                         0.0   
             2016-08-12                                         0.0   
             2016-08-12                                         0.0   
             2016-08-12                                         0.0   

                              1st_instar_ega  1st_instar_macrosteles  \
site   crop  collection_date                                           
Alvena Wheat 2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   

                              2nd_instar_ega  2nd_instar_macrosteles  \
site   crop  collection_date                                           
Alvena Wheat 2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   
             2016-08-12                  0.0                     0.0   

                              3rd_instar_ega  3rd_instar_ega_pre-alate  \
site   crop  collection_date                                             
Alvena Wheat 2016-08-12                  0.0                       0.0   
             2016-08-12                  0.0                       0.0   
             2016-08-12                  0.0                       0.0   
             2016-08-12                  0.0                       0.0   
             2016-08-12                  0.0                       0.0   

                              3rd_instar_macrosteles  4th_instar_macrosteles  \
site   crop  collection_date                                                   
Alvena Wheat 2016-08-12                          0.0                     0.0   
             2016-08-12                          0.0                     0.0   
             2016-08-12                          0.0                     0.0   
             2016-08-12                          0.0                     0.0   
             2016-08-12                          0.0                     0.0   

                              4th_instar_pre-alate  ...    thrips  \
site   crop  collection_date                        ...             
Alvena Wheat 2016-08-12                        0.0  ...         0   
             2016-08-12                        0.0  ...         0   
             2016-08-12                        0.0  ...         0   
             2016-08-12                        0.0  ...         0   
             2016-08-12                        0.0  ...         0   

                              total_alate_aphids  total_apterous_aphids  \
site   crop  collection_date                                              
Alvena Wheat 2016-08-12                      0.0                    0.0   
             2016-08-12                      0.0                    0.0   
             2016-08-12                      0.0                    0.0   
             2016-08-12                      0.0                    0.0   
             2016-08-12                      0.0                    0.0   

                              treehoppers  tychius_picirostris (weevil)  \
site   crop  collection_date                                              
Alvena Wheat 2016-08-12               0.0                           0.0   
             2016-08-12               0.0                           0.0   
             2016-08-12               0.0                           0.0   
             2016-08-12               0.0                

##### Distance

I've been concerned about redundancy and potential loss of precision due to the presence of a non-numerical value for `distance(m)`: "Combined".

- Can I be assured that the presence of "Combined" in the distance field always indicates the combination (summation, I presume) of all the other values at the same physical location, date, and of the same crop type?
- Conversely, if the non-combined (discrete) values do not account for the "Combined" value, can I be assured that there is no relationship — and therefore I should use both as separate samples?

To answer these questions, I'll separate discrete and aggregated sets, then examine:

- Equality of unique index records (site, crop, date).
- Equality of values in the "Combined" set and the corresponding sum of discrete sample values, as grouped by unique index.

In [141]:
s2_indexed_special_distance, s2_special_distance = (
    series[series['distance(m)'] != 'Combined'].sort_index()
    for series in (s2_indexed, s2))
s2_indexed_special_distance.index.size / s2_special_distance.index.size

1.0

In [138]:
s2_distance_combined = s2_indexed[s2_indexed['distance(m)'] == 'Combined'].sort_index()
s2_distance_discrete = s2_indexed[s2_indexed['distance(m)'] != 'Combined'].sort_index()

Quick sanity check by comparing index size:

In [139]:
s2_distance_discrete.index.drop_duplicates().size / s2_distance_combined.index.drop_duplicates().size

1.1375

That's to be expected, since many values tend to aggregate down to fewer values.

The actual unique difference, per set theory:

In [140]:
(s2_distance_combined.index ^ s2_distance_discrete.index).size

11

Oh? Eleven unexpected index entries? Are they in "discrete"? Maybe some rows were overlooked during aggregation, or maybe multiple sources were merged into one worksheet—some being aggregated and some not.

In [137]:
(s2_distance_combined.index ^ s2_distance_discrete.index).isin(s2_distance_discrete.index).size

11

All eleven are accounted for in "discrete" (but missing from "combined").

In [133]:
s2_discrete_unmatched = s2_distance_discrete.index.difference(s2_distance_combined.index)
s2_discrete_unmatched.to_frame()

,0
0,0
1,1
2,2
3,3
4,4
5,5
7,7
8,8
9,9
10,10


##### Compare Groups to Pre-existing "Combined"

I didn't expect that the data set from `Sheet2` would have sums (presumably from groupings of `distance(m)` values), yet also some non-aggregated values. Normally, these wouldn't be mixed, because they represent different dimensional orders. Because this data set's dimensional order is heterogeneous along the applicable indices, there could be some redundancy in the total information available, or contradictions.

Any redundancy, whether contradictory or not, would affect the calculation of sums for the for the intended [objective], unless there's exactly one record for each space and time combination — that is to say, if there are discrete values as well as previously "combined" values for the same point along the index, I'll have to avoid including the pre-calculated sum when aggregating my own sums, otherwise the resulting totals will be doubled.

[objective]: #Objective

In [63]:
s2_grouped = s2_indexed.groupby(s2_indexed.index.names + [s2_indexed['distance(m)'] == 'Combined'])

In [64]:
s2_grouped.size().describe()

count    165.000000
mean       4.048485
std        3.802769
min        1.000000
25%        1.000000
50%        2.000000
75%        6.000000
max       19.000000
dtype: float64

When grouped by all the indices, I hoped to see a `max = 1` for group size. Since the size some groups is greater than one, there are multiple values in some places. Therefore, it's worth checking for the consistency of values between discrete and aggregated for each combination of place, time, and specimen. If there are discrepencies, they'll need to be reconciled.

My sums and the source file's sums, side-by-side:

In [65]:
s2_distance_check = s2_grouped.sum().rename(index={False: 'Mine', True: 'Theirs'}, level='distance(m)').sum(axis='columns').unstack()
s2_distance_check#.head()

distance(m)                                   Mine   Theirs
site         crop         collection_date                  
Alvena       Barley       2016-06-10         193.0      NaN
                          2016-06-23         865.0    105.0
                          2016-06-24        1624.0     34.0
                          2016-07-08         562.0    137.0
                          2016-07-15        1833.0    404.0
                          2016-07-17         828.0     75.0
                          2016-07-22        1597.0     55.0
                          2016-07-28        1915.0     45.0
                          2016-08-05        1804.0   1363.0
                          2016-08-08         529.0     52.0
             Wheat        2016-06-10          85.0     27.0
                          2016-06-23         490.0    121.0
                          2016-07-15         884.0    324.0
                          2016-07-28        1131.0    206.0
                          2016-08-05        2195.0   2144.0
                          2016-08-12          21.0      6.0
                          2017-07-08         139.0     10.0
             Winter_Wheat 2016-07-15         700.0    295.0
Clavet       Barley       2016-06-24         385.0     74.0
Indian Head  Barley       2016-07-06         503.0     14.0
             Wheat        2016-07-19        1101.0    306.0
Kernan       Barley       2016-06-28        1408.0    103.0
                          2016-07-21        1691.0    314.0
                          2016-07-28        2466.0   1053.0
                          2016-08-05        1488.0    283.0
                          2017-07-08        3272.0     19.0
             Wheat        2016-07-21        2772.0    726.0
                          2016-08-05       11894.0  11059.0
                          2016-08-14         923.0     33.0
             Winter_Wheat 2016-06-28        1387.0    118.0
                          2016-07-21        1324.0    127.0
                          2016-07-28        2011.0    778.0
                          2016-08-05        1079.0    109.0
Kernan       Barley       2016-07-14        1495.0    154.0
             Wheat        2016-07-14        1385.0    350.0
Llewellyn    Barley       2016-06-10        1641.0      6.0
                          2016-06-20        2445.0     17.0
                          2016-07-15        3854.0     37.0
                          2016-07-21        1436.0     70.0
                          2016-08-05        1499.0     79.0
             Wheat        2016-06-10        1563.0      6.0
                          2016-07-11        1893.0     26.0
Melfort      Barley       2016-06-30        2954.0     16.0
                          2016-07-22        2006.0    149.0
                          2016-07-28        1096.0    154.0
             Oat          2016-06-10         344.0     13.0
                          2016-06-17        1715.0     45.0
                          2016-06-30        1812.0    117.0
                          2016-07-08        1449.0     75.0
                          2016-07-22        1449.0     60.0
                          2016-08-04        1454.0     48.0
             Wheat        2016-06-30        3314.0     16.0
                          2016-07-08        2903.0     41.0
Melfort      Barley       2016-08-05        2198.0    287.0
Outlook      Barley       2016-07-14        2709.0    468.0
                          2016-07-24        1773.0    259.0
             Wheat        2016-07-14        2181.0    266.0
                          2016-08-05        9500.0   6350.0
                          2016-08-09        3861.0   1891.0
                          2016-08-15        6073.0   1243.0
SEF          Barley       2016-06-06        5067.0      NaN
                          2016-06-10        2809.0      NaN
                          2016-06-23        2923.0      NaN
                          2016-07-05        3980.0      NaN
                          2016-07-11    

In [66]:
s2_distance_combined.eq(s2_distance_discrete)

,( damsel bug)nabis_americoferus_adult,1st_instar_ega,1st_instar_macrosteles,2nd_instar_ega,2nd_instar_macrosteles,3rd_instar_ega,3rd_instar_ega_pre-alate,3rd_instar_macrosteles,4th_instar_macrosteles,4th_instar_pre-alate,...,thrips,total_alate_aphids,total_apterous_aphids,treehoppers,tychius_picirostris (weevil),ufi_bugs,unnamed: 129,wasps_other,weevil,worms
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
